In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Non-std libs
import matplotlib.pyplot as plt

## Local libs
from scn_rrd import rrd_utils

References
- Unit is millisec.
    - https://github.com/librenms/librenms/blob/master/LibreNMS/Data/Source/FpingResponse.php :: `__construct()`
    - https://github.com/librenms/librenms/blob/fb3c2ecb1c9275108058e76e8af6f145a92235e7/LibreNMS/Polling/ConnectivityHelper.php :: `savePingStats()`

In [ ]:
locations = [
    'FCS',
    'sps-garfield',
    'occ-oromo',
]

devices_ports = rrd_utils.read_devices_ports()
deviceHostnames = devices_ports[ devices_ports['location'].isin(locations) ]['hostname'].unique()

In [ ]:
deviceHostname_to_df = {
    deviceHostname:
    rrd_utils.read_rrd(deviceHostname, rrd_utils.DEVICE_AGNOSTIC_RRD_FILENAMES.latency, '-6month')
    for deviceHostname in deviceHostnames
}

In [ ]:
fig, ax = plt.subplots()
for (deviceHostname, df) in deviceHostname_to_df.items():
    ax.plot(df['time'], df['ping'], label=deviceHostname)
ax.set_title('Latency')
ax.set_ylabel('millisec')
ax.legend()